<a href="https://colab.research.google.com/github/LCaravaggio/scrapers/blob/master/Nueva_P%C3%A1gina_de_Coto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -U seleniumbase
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!sudo dpkg -i google-chrome-stable_current_amd64.deb
!sudo apt-get install -f

In [2]:
from seleniumbase import Driver
from seleniumbase import page_actions

In [97]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

def scrapcoto(sitecoto, driver):
    try:
        driver.get(sitecoto)
        wait = WebDriverWait(driver, 12)  # Esperar hasta 12s antes de fallar

        # Intentar obtener el nombre del producto
        try:
            nombre = wait.until(EC.presence_of_element_located((By.XPATH,
                '//app-product-detail//aside//h2'))).text
        except:
            nombre = "N/A"

        # Intentar obtener el precio
        xpaths_precio = [
            '//app-product-detail/div/aside/div/article/div[2]/div[2]',
            '//app-product-detail/div/aside/div/article/div[2]/div[3]',
            '//app-product-detail//aside//div/article/div[2]/div[2]',
            '//app-product-detail//aside//div/article/div[3]/div[2]/div[2]/var',
            '//app-product-detail//aside//div/article/div[2]/div[2]/div[3]/var'
        ]

        precio = "N/A"
        for xpath in xpaths_precio:
          try:
            elemento = driver.find_element(By.XPATH, xpath)
            texto = elemento.text
            if "Precio contado :" in texto:
              precio = texto
              break  # Detener el bucle una vez encontrado

          except:
            continue  # Si no encuentra el elemento, pasa al siguiente

        # Limpiar el precio

        precio = re.sub(r"[\$\s\n\r\t]|PRECIOCONTADO|PRECIOREGULAR|Precio contado :|p1Klgm|p1K|p1Klgmsu", "", precio)

        return f"{nombre};{precio}"

    except Exception as e:
        return f"Error."


In [78]:
import sys
import datetime

def corre(lista, nombre):
  a=""

  start_time = time.time()
  max_time = 6 * 60 * 60
  bucle=1
  driver = Driver(uc=True)

  for l in lista:
    if (bucle) % 100 == 0:
      driver = Driver(uc=True)

    a+= l + ";"

    sys.stdout.write(f"\rProcesando... {bucle}/{len(lista)+1}")
    sys.stdout.flush()
    bucle+=1

    a+=scrapcoto(l, driver)

    a+="\n"

    if time.time() - start_time > max_time:
        print("Luego de 6 horas de intentos, el loop se interrumpe.")
        break

  sys.stdout.write("\r" + " " * 30 + "\r")
  sys.stdout.flush()
  sys.stdout.write(f"Proceso finalizado.")
  sys.stdout.flush()

  now = datetime.datetime.now()
  nw=str(now.strftime("%Y-%m-%d %H-%M-%S"))

  nombre='/content/' + str(nombre) + ' ' + nw + '.csv'
  with open(nombre, 'w', newline="\n", encoding='utf-8') as f:
    f.write(a.replace('\u2022', '*'))
    f.close

  sys.stdout.write("\r" + " " * 30 + "\r")
  sys.stdout.flush()
  sys.stdout.write(f"Se generó el archivo: {nombre}")
  sys.stdout.flush()

In [101]:
with open('/content/coto.txt') as file:

    lista = [line.rstrip() for line in file]
    corre(lista, 'coto')

Se generó el archivo: /content/coto 2025-01-29 21-35-02.csv